# Creating a Deep Learning Forecasting model with PyTorch
In this notebook, you will create a Recurrent Neural Network (RNN) that you can use to forecast the battery cycles used for time series battery data.

In [2]:
# Install the necessary libraries directly into the notebook context

dbutils.library.installPyPI('torch')
dbutils.library.restartPython()
dbutils.library.list()

Out[ 1 ]: ['PyPI:(torch)-(empty)-(empty)-(empty)']

In [3]:
import uuid
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1) # ensure repeatability

import torch
import torch.nn as nn
import torch.optim as optim

pd.set_option('display.max_columns', 10)

torch.manual_seed(0) # ensure repeatability

Out[ 1 ]: <torch._C.Generator at 0x7f6a28f6c8b0>

### Download the data

The following cell will download the data set containing the daily battery time series.
The CSV file will be saved in a temporary folder on the Databricks cluster for later reuse.

In [5]:
# Create a temporary folder to store locally relevant content for this notebook
tempFolderName = '/FileStore/CarBatteries_{0}'.format(uuid.uuid4())
dbutils.fs.mkdirs(tempFolderName)
print('Content files will be saved to {0}'.format(tempFolderName))

filesToDownload = ['daily-battery-time-series-v2.csv']

for fileToDownload in filesToDownload:
  downloadCommand = 'wget -O ''/dbfs{0}/{1}'' ''https://databricksdemostore.blob.core.windows.net/data/connected-car/{1}'''.format(tempFolderName, fileToDownload)
  print(downloadCommand)
  ret_val = os.system(downloadCommand)
  print('System resturned value %s' % (ret_val))

Content files will be saved to /FileStore/CarBatteries_7ec54f66-baed-4639-80b4-15d97242775b
wget -O /dbfs/FileStore/CarBatteries_7ec54f66-baed-4639-80b4-15d97242775b/daily-battery-time-series-v2.csv https://databricksdemostore.blob.core.windows.net/data/connected-car/daily-battery-time-series-v2.csv
System resturned value 0

A simple check to make sure our data file was correctly downloaded.

In [7]:
# Check that all files are successfully downloaded
dbutils.fs.ls(tempFolderName)

Out[ 3 ]: [FileInfo(path='dbfs:/FileStore/CarBatteries_7ec54f66-baed-4639-80b4-15d97242775b/daily-battery-time-series-v2.csv', name='daily-battery-time-series-v2.csv', size=132341)]

### Load the data

The previously downloaded CSV file will be loaded into a Pandas Dataframe and its first few rows inspected.

In [9]:
# Load the dataset
file_path = '/dbfs%s/%s' % (tempFolderName, filesToDownload[0])
df = pd.read_csv(file_path, delimiter=',')
df = df[['Date','Battery_ID','Battery_Age_Days','Number_Of_Trips','Daily_Trip_Duration','Daily_Cycles_Used', 'Lifetime_Cycles_Used', 'Battery_Rated_Cycles']]

# Remove the temporary folder
dbutils.fs.rm(tempFolderName, recurse=True)

# Inspect the data frame
df.head()

,Date,Battery_ID,Battery_Age_Days,Number_Of_Trips,Daily_Trip_Duration,Daily_Cycles_Used,Lifetime_Cycles_Used,Battery_Rated_Cycles
0,2013-01-01,0,0,5,67.845608,0.166920,0.166920,200.0
1,2013-01-02,0,1,5,53.450798,0.131505,0.298425,200.0
2,2013-01-03,0,2,5,58.841433,0.144767,0.443193,200.0
3,2013-01-04,0,3,5,60.638403,0.149188,0.592381,200.0
4,2013-01-05,0,4,5,62.646910,0.154130,0.746511,200.0


The time series related to one specific Battery_ID will be isolated and its shape checked. To keep the model simple and make is easier to understand, only one column will be used - Daily_Cycles_Used.

In case the dataset contains more time series, the process of training and prediction must be repeated for each individual series.

In [11]:
# Isolate the time series related to one Battery_ID
df_source = df[df['Battery_ID'] == 0][['Daily_Cycles_Used']]
df_source.shape

Out[ 5 ]: (1539, 1)

Reshape our input as a one dimensional array. This will make some of the operations we'll perform easier to follow and understand.

In [13]:
source = df_source.values.reshape(1, df_source.shape[0])[0]

### Prepare the data

Our time series prediction model will use a special kind of RNN (Recurrent Neural Network) built out of LSTM (Long Short Term Memory) cells. LSTMs are not particularly happy with very long series so we are setting the maximum limit of a time series to 250 steps (```sample_size```). Based on this value, we calculate then the maximum number of non-overlapping samples we can get from our original time series (```num_samples```).

We then consolidate these samples into two matrixes, ```input``` and ```output```. Notice they are built in a way that for every element Xn in every sample in ```input```, the corresponding element from ```output``` is equal to the one that follows Xn (which is Xn+1). The fundamental idea is that we're looking to train a model that will be capable of predicting Xn+1 based on Xn.

In [15]:
sample_size = 250
num_samples = source.shape[0] // sample_size

input = np.zeros((num_samples, sample_size))
output = np.zeros((num_samples, sample_size))

for i in range(num_samples):
  input[i] = source[-(i+1) * sample_size - 2 : -i * sample_size - 2]
  output[i] = source[-(i+1) * sample_size - 1 : -i * sample_size - 1]

Since we are using PyTorch, we're moving ```input``` and ```output``` into tensor space. The \_t notation is used to identify a variable that is a tensor.

In [17]:
input_t = torch.from_numpy(input)
target_t = torch.from_numpy(output)
print(input_t.shape)
print(target_t.shape)

torch.Size([6, 250])
torch.Size([6, 250])

### Build and train the model

Next, we define our model as a class derived from the base class ```nn.Module```. Our model contains two hidden LSTM layers of sizes ```hidden_layer1_size``` and ```hidden_layer2_size``` respectively. The output of the second LSTM layer is fed into a linear layer that will combine all components into a single output.

Each hidden layer also needs a pair of variables to hold internal state (```h_t```,```c_t``` and ```h_t2```,```c_t2``` respectively). They are used by the LSTM cells to keep track of their "memory" during the run of every epoch (implemented by the ```forward``` method). Notice how the internal state is reset at the beginning of each epoch run.

Also notice the ```future``` parameter which controls whether we want to also make predictions into the future or not. The value of this parameter will be 0 during the training process and set to a number of days when the model is called to make a prediction once it is trained.

In [19]:
# The number of nodes in the hidden layers
hidden_layer1_size = 75
hidden_layer2_size = 75

class LSTMPredictor(nn.Module):
    def __init__(self):
        super(LSTMPredictor, self).__init__()
        self.lstm1 = nn.LSTMCell(1, hidden_layer1_size)
        self.lstm2 = nn.LSTMCell(hidden_layer1_size, hidden_layer2_size)
        self.linear = nn.Linear(hidden_layer2_size, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), hidden_layer1_size, dtype=torch.double)
        c_t = torch.zeros(input.size(0), hidden_layer1_size, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), hidden_layer2_size, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), hidden_layer2_size, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

Set the number of epochs, the learning rate, the method to calculate the loss function, and the optimizer used for the backwards pass on the network during training.

In [21]:
# Increase the number of epochs for better results
epochs = 150
learning_rate = 0.5

# build the model
pred = LSTMPredictor()
pred.double() #convert all internal values to doubles
criteria = nn.MSELoss()
optimizer = torch.optim.Adam(pred.parameters(), lr=learning_rate)

Perform the actual training on the model.

For each epoch, we will perform the following steps:

- Make a prediction using the ```input_t``` input tensor
- Calculate how far is the predicted result from the expected result (stored in the ```target_t``` tensor). The distance is given by the value of the loss function, which we also save.
- Zero out the gradients (we are resetting them on each epoch)
- Trigger the backpropagation process through which we are recalibrating the internal weights of the network
- Activate the optimizer to help the recalibration process

In [23]:
losses = []

for epoch in np.arange(1, epochs + 1):
    
    print('Starting epoch %s...' % (epoch))
    
    # Feed the input through the network
    out = pred(input_t)

    # Calculate loss tensor
    loss = criteria(out, target_t)
    losses += [loss.item()]
    print('Current loss: %s' % (loss.item()))
    
    optimizer.zero_grad()
    
    # Trigger backpropagation
    loss.backward()
    # Move on
    optimizer.step()

Starting epoch 1...
Current loss: 0.021775809198469307
Starting epoch 2...
Current loss: 471.2960242948518
Starting epoch 3...
Current loss: 52.097640831602725
Starting epoch 4...
Current loss: 53.6799114008337
Starting epoch 5...
Current loss: 234.56473798341688
Starting epoch 6...
Current loss: 217.4136138260876
Starting epoch 7...
Current loss: 34.9357771709064
Starting epoch 8...
Current loss: 3.019658577680696
Starting epoch 9...
Current loss: 0.25136909526042883
Starting epoch 10...
Current loss: 1.1700782756376753
Starting epoch 11...
Current loss: 0.002944327185952843
Starting epoch 12...
Current loss: 0.02120753290484105
Starting epoch 13...
Current loss: 0.05175410549508887
Starting epoch 14...
Current loss: 0.08981591144032687
Starting epoch 15...
Current loss: 0.13155633755194512
Starting epoch 16...
Current loss: 0.17390612232237276
Starting epoch 17...
Current loss: 0.21445608970998478
Starting epoch 18...
Current loss: 0.25137075839361833
Starting epoch 19...
Current loss: 0.2833152833440504
Starting epoch 20...
Current loss: 0.30939107689168194
Starting epoch 21...
Current loss: 0.3290772652327771
Starting epoch 22...
Current loss: 0.34217630107793745
Starting epoch 23...
Current loss: 0.34876281524133396
Starting epoch 24...
Current loss: 0.3491352902607125
Starting epoch 25...
Current loss: 0.34377046290746527
Starting epoch 26...
Current loss: 0.3332805632874005
Starting epoch 27...
Current loss: 0.31837361119127455
Starting epoch 28...
Current loss: 0.2998170393433731
Starting epoch 29...
Current loss: 0.2784049152830253
Starting epoch 30...
Current loss: 0.2549290015370592
Starting epoch 31...
Current loss: 0.2301538374164846
Starting epoch 32...
Current loss: 0.20479595319564803
Starting epoch 33...
Current loss: 0.17950724523327305
Starting epoch 34...
Current loss: 0.15486245435584436
Starting epoch 35...
Current loss: 0.13135060427855721
Starting epoch 36...
Current loss: 0.10937017598024412
Starting epoch 37...
Current loss: 0.0892277210848553
Starting epoch 38...
Current loss: 0.07113955508397504
Starting epoch 39...
Current loss: 0.05523612166631631
Starting epoch 40...
Current loss: 0.0415685838639429
Starting epoch 41...
Current loss: 0.030117176912284195
Starting epoch 42...
Current loss: 0.020800851770021063
Starting epoch 43...
Current loss: 0.013487746693184258
Starting epoch 44...
Current loss: 0.008006046110513584
Starting epoch 45...
Current loss: 0.004154819842084808
Starting epoch 46...
Current loss: 0.0017144795914006534
Starting epoch 47...
Current loss: 0.0004565414803103086
Starting epoch 48...
Current loss: 0.00015244085354047395
Starting epoch 49...
Current loss: 0.0005812062392574989
Starting epoch 50...
Current loss: 0.0015358608209278032
Starting epoch 51...
Current loss: 0.0028284797840690897
Starting epoch 52...
Current loss: 0.00429388839213348
Starting epoch 53...
Current loss: 0.005792036848474397
Starting epoch 54...
Current loss: 0.0072091324890712835
Starting epoch 55...
Current loss: 0.008457646576128193
Starting epoch 56...
Current loss: 0.009475341274559386
Starting epoch 57...
Current loss: 0.010223482033172061
Starting epoch 58...
Current loss: 0.010684411673044528
Starting epoch 59...
Current loss: 0.010858665455275343
Starting epoch 60...
Current loss: 0.01076180199359038
Starting epoch 61...
Current loss: 0.010421114058218637
Starting epoch 62...
Current loss: 0.00987236721699559
Starting epoch 63...
Current loss: 0.009156694111894738
Starting epoch 64...
Current loss: 0.008317749248758539
Starting epoch 65...
Current loss: 0.007399204741778851
Starting epoch 66...
Current loss: 0.00644264268949029
Starting epoch 67...
Current loss: 0.005485875839450441
Starting epoch 68...
Current loss: 0.0045617058501289595
Starting epoch 69...
Current loss: 0.003697108533827679
Starting epoch 70...
Current loss: 0.002912818529287357
Starting epoch 71...
Current loss: 0.002223272280112227
Starting epoch 72...
Current loss: 0.0016368581698311608
Starting epoch 73...
Current loss:

Display the evolution of the loss function. We would expect the graph to flatline after a few initial pulses.

In [25]:
fig = plt.figure(figsize=(30,10))
plt.title('The evolution of the LOSS function during training', fontsize=10)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(np.arange(epochs), losses, 'r', linewidth=1.0)

display(fig)
plt.close()

### Predict the future

One the training process is finished, we are using the trained model to predict the values for the next 30 days. Since our sample size in ```sample_size``` we are just taking the last ```sample_size``` elements from the original time series and feed them to the model.

Notice the ```with torch.no_grad()``` option which basically tells PyTorch this is not part of any training process, hence there is no need to track the gradients on the tensors involved.

In [27]:
# The model is trained, predict the next 30 days
days_to_predict = 30

# Get the tensor with the last sample_size values
final_input = torch.from_numpy(source[-sample_size:].reshape(1, sample_size))

# No need to track gradient anymore
with torch.no_grad():
    
    y_t = pred(final_input, future=days_to_predict)
    y = y_t.detach().numpy()

The result of the prediction will contain the predicted output corresponding to the input plus a number of elements equal to the number of future days we need prediction for.

We'll just take a look at the future values predicted.

In [29]:
future_predictions = y[0, - days_to_predict:]
print(future_predictions)

[0.14263596 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596
 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596
 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596
 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596
 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596 0.14263596]

In [ ]:
fig = plt.figure(figsize=(30,10))
plt.title('Predict future values \n(Red values are predicted values)', fontsize=10)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(np.arange(sample_size), source[-sample_size:], 'g', linewidth=1.0)
plt.plot(np.arange(sample_size, sample_size + days_to_predict), future_predictions, 'r', linewidth=1.0)

display(fig)
plt.close()

Plot the last ```sample_size``` elements from the original time series in green and the predicted values for the next 30 days in red.

Please note that we are using synthetic training data and the target value was randomly generated around a mean, thus you will observe that the predictions are closer to the mean of the dataset.